# Extract data from BMRS

- Extract the curtailed amount and costs for a list of windfarms from the BMRS service
- The notebook requires a file in the folder which only contains the APIKey for the BMRS service and which needs to be called APIKey

In [ ]:
import httplib2
import pandas as pd
from datetime import date, datetime, timedelta, timezone
import xml.etree.ElementTree as ET

In [ ]:
# Function to download the data
def post_elexon(url, print_response=False, print_content=False):
    http_obj = httplib2.Http()
    resp, content = http_obj.request(
        uri=url,
        method='GET',
        headers={'Content-Type': 'application/xml; charset=UTF-8'},
    )

    if print_response:
        print('===Response===')
        print(pformat(resp))
    
    if print_content:
        print('===Content===')
        print(pformat(content))
        
    if print_response or print_content:
        print('===Finished===')
    return content

In [ ]:
# The notebook requires the APIKey file with the APIKey
try:
    with open('APIKey') as f:
        lines = f.readlines()
    APIKey = lines[0]
except:
    print("No APIKey file")

## Extract the data for one settlement date and period

It prints all BM units IDs and the choosen value

In [ ]:
report_name = "DERBMDATA"
SettlementDate = "2018-02-10"
Period = "8"
ServiceType = "xml"

# This extracts all units for a specific period
url_name = "https://api.bmreports.com/BMRS/" + report_name + "/v1?APIKey=" + APIKey + "&SettlementDate=" + SettlementDate + \
            "&SettlementPeriod=" + Period + "&ServiceType=" + ServiceType

content = post_elexon(
    url=url_name,
    print_response=False,
    print_content=False
)

# Parse into an ElementTree
root = ET.fromstring(content)
tree = ET.ElementTree(root)

# Write to file for checking
tree.write('constraint.xml')

# Extract values for one of the following categories
categories = ["bav", "oav", "ipbav", "ipoav", "ipbc", "ipoc"]
s = "./responseBody/" + categories[4] + "/responseList/item"
       
# Find all BM units
for item in root.findall(s):
    bmUnitID = item.find("bmUnitID").text
    total = item.find("total").text
    print(bmUnitID, total)

## Loop over a range of dates

- Extract the choosen value for a range of dates and BM units

In [ ]:
# Import database with time zones
# Set UK timezone
from dateutil import tz
uk_tz = tz.gettz('Europe/London')

# Function to define a date range
def daterange(start_date, end_date, inclusive=True):
    add_day = 0
    if inclusive:
        add_day = 1
    for n in range(int((end_date - start_date).days) + add_day):
        yield start_date + timedelta(n)

In [ ]:
# Set dates to investigate
start_date = date(2020, 1, 2)
end_date = date(2020, 1, 2)

# Set list of BM units for which to extract the data
BMunits = ["T_WHILW-1", "T_WHILW-2", "T_BLLA-1", "T_BLLA-2", "T_CLDCW-1", "T_CLDNW-1", "T_CLDSW-1", "T_CRYRW-2", "T_CRYRW-3", \
           "T_HADHW-1", "T_KLGLW-1"]
    
# Extract values for one generator
categories = ["bav", "oav", "ipbav", "ipoav", "ipbc", "ipoc"]

# List to store the data    
data = list()
    
# Loop over the complete date range    
for single_date in daterange(start_date, end_date):
    print(single_date.strftime("%Y-%m-%d"))
    
    # Loop over all periods from 1 to 50
    for period in range(1, 51):
        #print(single_date.strftime("%Y-%m-%d"), period)
        SettlementDate = single_date.strftime("%Y-%m-%d")
        # This extracts all units for a specific period
        url_name = "https://api.bmreports.com/BMRS/" + report_name + "/v1?APIKey=" + APIKey + "&SettlementDate=" + SettlementDate + \
                    "&SettlementPeriod=" + str(period) + "&ServiceType=" + ServiceType

        content = post_elexon(
            url=url_name,
            print_response=False,
            print_content=False
        )
                
        # Take the starting datetime, convert it to UTC and add 30 minutes for each period
        date_uk = datetime.strptime(single_date.strftime('%Y%m%d'), '%Y%m%d')
        date_uk = date_uk.replace(tzinfo=uk_tz)
        date_utc = date_uk.astimezone(timezone.utc)
        date_utc = date_utc + timedelta(minutes=(period - 1)*30)
        #print(date_utc.isoformat())        
        #date_uk = date_utc.astimezone(uk_tz) 
        #print(date_uk.isoformat())

        # Parse into an ElementTree
        root = ET.fromstring(content)
        tree = ET.ElementTree(root)
        
        # Write to file for checking
        if False:
            s = 'output' + str(period) + '.xml'
            tree.write(s)
        
        # Check if data found
        if str(root.find("./responseMetadata/description").text) == "Success":
            #print('Period:', period)
            
            curtailed_energy = dict((el, 0) for el in BMunits)
            curtailed_payment = dict((el, 0) for el in BMunits)
    
            # Find amount of curtailed energy in MWh           
            s = "./responseBody/" + categories[2] + "/responseList/item"
            data_list = [date_utc]
            for item in root.findall(s):               
                bmUnitID = item.find("bmUnitID").text
                # Check for a specific unit
                for unit in BMunits:
                    curtailed = 0
                    if str(bmUnitID) == unit:
                        dataType = item.find("dataType").text
                        if str(dataType) == "Original":
                            curtailed += float(item.find("total").text)
                            #print(bmUnitID, "Curtailed =", curtailed)
                            curtailed_energy[unit] = curtailed

            # Find amount of money paid for curtailed energy in £
            s = "./responseBody/" + categories[4] + "/responseList/item"
            payment = 0
            for item in root.findall(s):
                bmUnitID = item.find("bmUnitID").text
                # Check for a specific unit
                for unit in BMunits:
                    payment = 0
                    if str(bmUnitID) == unit:
                        payment += float(item.find("total").text)
                        #print(bmUnitID, "Payment =", payment)
                        curtailed_payment[unit] = payment

            # Append data for this date and period to the list
            #data.append([date_utc, curtailed_energy, payment])
            for key, value in curtailed_energy.items():
                temp = [key,value]
                data_list.append(value)
                
            for key, value in curtailed_payment.items():
                temp = [key,value]
                data_list.append(value)
            
            data.append(data_list)  
        
#print(data)
columns = ['Dates_[UTC]']
for unit in BMunits:
    columns.append(unit + '_Energy_[MWh]')
for unit in BMunits:
    columns.append(unit + '_Payment_[GBP]')

# Convert to a dataframe and write to a csv file
df = pd.DataFrame(data, columns=columns).set_index('Dates_[UTC]') 
df.to_csv('Scottish_curtailed_wind.csv')

In [ ]:
# Plot for checking
df["T_WHILW-1_Energy_[MWh]"].plot()

In [ ]:
# Plot for checking
df["T_WHILW-1_Payment_[GBP]"].plot()